In [152]:
from http.client import responses

import pandas as pd
import numpy as np
import folium
import requests
from argon2 import Parameters

from bs4 import BeautifulSoup
from prompt_toolkit.formatted_text import OneStyleAndTextTuple

# pip install tqdm : progressBar 구현
# from tqdm.notebook import tqdm
from tqdm import tqdm
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# jupyter nbconvert --to script coffeeStore.ipynb

In [153]:
# 크롬 드라이버 다운로드 :  https://googlechromelabs.github.io/chrome-for-testing/
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간 10초
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [154]:
# driver.maximize_window()

In [155]:
# 공용 변수 선언
dataOut = './../dataOut/'

In [156]:
# 스타벅스) 매장 찾기-지역 검색 클릭
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(starbucks_url) # 해당 페이지로 이동하기

In [157]:
# 스타벅스 '서울' 링크 클릭
starbucks_seoul_selector = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_selector))).click()

In [158]:
# 스타벅스 '서울'-'전체' 링크 클릭
starbucks_seoul_all = '#mCSB_2_container > ul > li:nth-child(1) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all))).click()

In [159]:
# 스타벅스 html 코드를 파싱해서 html 파일에 기록합니다.
html = driver.page_source # 해당 페이지 소스 코드 반환
filename = 'starbucks.html'
htmlFile = open(filename, mode='wt', encoding='UTF-8')
print(html, file=htmlFile)
htmlFile.close()
print(filename, '파일 생성됨')

starbucks.html 파일 생성됨


In [160]:
# 파싱된 결과를 BeautifulSoup 객체로 생성
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [161]:
# id 속성이 mCSB_3_container인 <div> 태그를 찾아 주세요.
container = soup.find('div', attrs={'id': 'mCSB_3_container'})

# 그 아래에 있는 모든 <li> 태그를 찾아 주세요.
storeAll = container.find_all('li')

print(f'모든 매장 갯수 : {len(storeAll)}')

모든 매장 갯수 : 666


In [162]:
starbucksData = [] # 스타벅스 매장 리스트

for store in storeAll:
    # print(store)
    brand = '스타벅스'
    name = store.find('i').text.strip()
    address = store.find('p').text.replace('1522-3232', '')
    imsi = address.split(' ')
    sido = imsi[0]
    gungu = imsi[1]
    latitude = store['data-lat']
    longitude = store['data-long']

    starbucksData.append((brand, name, address, sido, gungu, latitude, longitude))
    # break
# end for

print(f'스타벅스 매장 갯수 : {len(starbucksData)}')
# print(starbucksData)

스타벅스 매장 갯수 : 666


In [163]:
'''
<li class="quickResultLstCon"
    data-code="3762"
    data-hlytag="null"
    data-index="0"
    data-lat="37.501087"
    data-long="127.043069"
    data-name="역삼아레나빌딩"
    data-storecd="1509"
    style="background:#fff">

    <strong data-my_siren_order_store_yn="N"
            data-name="역삼아레나빌딩"
            data-store="1509"
            data-yn="N">
        역삼아레나빌딩
    </strong>

    <p class="result_details">
        서울특별시 강남구 언주로 425 (역삼동)<br/>
        1522-3232
    </p>

    <i class="pin_general">리저브 매장 2번</i>

</li>

'''

'\n<li class="quickResultLstCon"\n    data-code="3762"\n    data-hlytag="null"\n    data-index="0"\n    data-lat="37.501087"\n    data-long="127.043069"\n    data-name="역삼아레나빌딩"\n    data-storecd="1509"\n    style="background:#fff">\n\n    <strong data-my_siren_order_store_yn="N"\n            data-name="역삼아레나빌딩"\n            data-store="1509"\n            data-yn="N">\n        역삼아레나빌딩\n    </strong>\n\n    <p class="result_details">\n        서울특별시 강남구 언주로 425 (역삼동)<br/>\n        1522-3232\n    </p>\n\n    <i class="pin_general">리저브 매장 2번</i>\n\n</li>\n\n'

In [164]:
sbColumns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도']
sbDataFrame = pd.DataFrame(starbucksData)
sbDataFrame.columns = sbColumns
sbDataFrame.head()

,브랜드,상호,주소,시도,군구,위도,경도
0,스타벅스,리저브 매장 2번,서울특별시 강남구 언주로 425 (역삼동),서울특별시,강남구,37.501087,127.043069
1,스타벅스,리저브 매장 2번,서울특별시 강남구 강남대로 538 (논현동),서울특별시,강남구,37.510178,127.022223
2,스타벅스,리저브 매장 2번,서울특별시 강남구 강남대로 584 (논현동),서울특별시,강남구,37.5139309,127.0206057
3,스타벅스,리저브 매장 2번,서울특별시 강남구 테헤란로 125 (역삼동),서울특별시,강남구,37.499517,127.031495
4,스타벅스,리저브 매장 2번,서울특별시 강남구 남부순환로 2947 (대치동),서울특별시,강남구,37.494668,127.062583


In [165]:
sbFilename = dataOut + 'starbucks_file.csv'
sbDataFrame.to_csv(sbFilename, index=False, encoding='UTF-8')
print(f'{sbFilename} 파일이 저장되었습니다.')

./../dataOut/starbucks_file.csv 파일이 저장되었습니다.


In [166]:
# 위도, 경도 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {sbDataFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {sbDataFrame["경도"].isnull().sum()}')

위도 누락 데이터 갯수 : 0
경도 누락 데이터 갯수 : 0


In [167]:
print('# 서울시 구 갯수 파악')
seoul_gu_list = list(sbDataFrame['군구'].unique())
print(f'서울시 구 갯수: {len(seoul_gu_list)}')
print(f'\n서울시 구 목록 : {seoul_gu_list}')

# 서울시 구 갯수 파악
서울시 구 갯수: 25

서울시 구 목록 : ['강남구', '강북구', '강서구', '관악구', '광진구', '금천구', '노원구', '도봉구', '동작구', '마포구', '서대문구', '서초구', '성북구', '송파구', '양천구', '영등포구', '은평구', '종로구', '중구', '강동구', '구로구', '동대문구', '성동구', '용산구', '중랑구']


In [168]:
# 누락된 위도와 경도 정보는 kakao api를 이용하여 채워 넣도록 합니다.
# https://developers.kakao.com/
# kakao에 로그인하여 API 키 발급 받기
# 주의) 카카오 개발자 사이트에서 카카오맵-사용설정-On 상태로 변경해주세요.

In [169]:
url_header = 'https://dapi.kakao.com/v2/local/search/address.json?query='
api_key = 'a27595520f0c90739026b9f6ace4279c'
header = {'Authorization': 'KakaoAK ' + api_key}

In [170]:
# 주소를 입력 받아서 위도와 경도를 반환해주는 함수 구현하기
def getGeoCoder(address):
    result = ''
    url = url_header + address
    response = requests.get(url, headers=header)

    if response.status_code == 200:
        try:
            # print(response.json())
            jsondata = response.json()["documents"][0]["address"]
            return jsondata['y'], jsondata['x']
        except Exception as err:
            print(err)
            return None
        # end try
    else:
        result = f'Error[{response.status_code}]'
    # end if

    return result
# end def getGeoCoder

In [171]:
# 다음 주소에 대한 테스트를 진행합니다.
address01 = '서울 중랑구 망우로 460 (망우동)'
geoInfo01 = getGeoCoder(address01) # 잘 동작하는 주소
print(geoInfo01)

address02 = '서울 노원구 한글비석로 409 (상계동) 1~2층'
geoInfo02 = getGeoCoder(address02) # NoneType이 리턴되는 주소
geoInfo02

('37.6001065609187', '127.103136691889')
list index out of range


In [172]:
'''
케이스 01
panLatTo('127.079680860838','37.5879908042663','6');fnMove();

케이스 02
panLatTo('0','0','11');fnMove();

케이스 03
panAddTo('서울 중랑구 신내로 211 (신내동)','8');fnMove();
'''


"\n케이스 01\npanLatTo('127.079680860838','37.5879908042663','6');fnMove();\n\n케이스 02\npanLatTo('0','0','11');fnMove();\n\n케이스 03\npanAddTo('서울 중랑구 신내로 211 (신내동)','8');fnMove();\n"

In [173]:
# 이디야 매장 정보
ediya_url = 'https://www.ediya.com/contents/find_store.html'
driver.get(ediya_url)

In [174]:
# 이디야 '매장명'과 '주소' 검색에서 '주소' 버튼 클릭
ediya_address_selector = '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_address_selector))).click()

In [175]:
# 서울시 구 목록을 순환하면서 크롤링
ediyaData = [] # 이디야 매장 정보 리스트
incorrect_address = [] # 위도/경도 추출에 문제가 있는 주소 리스트

ediya_search_inputbox = '#keyword' # 이디야 주소 검색 입력란
ediya_search_button = '#keyword_div > form > button' # 이디야 주소 검색 (돋보기) 버튼

for gu in tqdm(seoul_gu_list):
    # 주소 검색 입력란 초기화
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_inputbox))).clear()

    # 주소 검색어 입력 : 예시) 서울 강남구
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_inputbox))).send_keys(f'서울 {gu}')

    # 찾기(돋보기) 버튼 클릭
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_button))).click()

    # 이디야 매장 정보 수집
    html = driver.page_source

    if gu == '마포구': # 하나의 구 정보만 저장해보기
        gufile = open(f'서울 {gu}.html', 'wt', encoding='UTF-8')
        print(html, file=gufile)
        gufile.close()
        print(f'서울 {gu}.html 파일 생성 완료')
    # end if

    soup = BeautifulSoup(html, 'html.parser')

    # 매장 정보는 id 속성이 placesList인 <ul> 태그 아래에 있습니다.
    ul_tag = soup.find('ul', id='placesList')

    oneGuStoreList = ul_tag.find_all('li') # 구별 매장 리스트

    for store in oneGuStoreList:
        # print(store)
        brand = '이디야'
        name = store.find('dt').text.strip()
        address = store.find('dd').text
        imsi = address.split(' ')
        sido = imsi[0]
        gungu = imsi[1]

        # 위도와 경도는 잠시
        geoInfoString = store.find('a')['onclick']
        # print(geoInfoString)

        latitude = np.nan # 결측치
        longitude = np.nan

        # 위도와 경도 보정
        if geoInfoString.startswith('panLatTo') :
            if geoInfoString.startswith("panLatTo('0','0'") : # 잘못된 위도/경도
                # address를 사용하여 Kakao API에서 물어 보기
                geo_info = getGeoCoder(address)
                if geo_info != None:
                    latitude, longitude = geo_info
                else:
                    incorrect_address.append(address)
                    latitude = None
                    longitude = None
                # end if
            else: # 올바른 위도/경도
                latLong = geoInfoString.split("'")
                latitude = latLong[3]
                longitude = latLong[1]
                # print(f'위도 : {latitude}, 경도 : {longitude}')
            # end if

        else: # 'panAddTo'으로 시작하는 경우
            # address를 사용하여 Kakao API에서 물어 보기
            geo_info = getGeoCoder(address)
            if geo_info != None:
                latitude, longitude = geo_info
            else:
                incorrect_address.append(address)
                latitude = None
                longitude = None
            # end if
        # end if

        ediyaData.append((brand, name, address, sido, gungu, latitude, longitude))

        # break # 주의) 차후 삭제 예정
    # end inner for

    # break # 주의) 차후 삭제 예정
# end outer for

print(f'이디야 매장 갯수 : {len(ediyaData)}')

 36%|███▌      | 9/25 [00:13<00:25,  1.61s/it]

서울 마포구.html 파일 생성 완료


 40%|████      | 10/25 [00:15<00:26,  1.77s/it]

list index out of range


 64%|██████▍   | 16/25 [00:28<00:17,  1.95s/it]

list index out of range


100%|██████████| 25/25 [00:46<00:00,  1.86s/it]

이디야 매장 갯수 : 462


In [176]:
'''
<li class="item">
    <a href="#c" onclick="panLatTo('0','0','0'); fnMove();">
        <div class="store_thum">
            <img src="../images/customer/store_thum.gif" />
        </div>
        <dl>
            <dt>강남YMCA점</dt>
            <dd>서울 강남구 논현동</dd>
        </dl>
    </a>
</li>
'''

'\n<li class="item">\n    <a href="#c" onclick="panLatTo(\'0\',\'0\',\'0\'); fnMove();">\n        <div class="store_thum">\n            <img src="../images/customer/store_thum.gif" />\n        </div>\n        <dl>\n            <dt>강남YMCA점</dt>\n            <dd>서울 강남구 논현동</dd>\n        </dl>\n    </a>\n</li>\n'

In [177]:
edColumns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도']
edDataFrame = pd.DataFrame(ediyaData)
edDataFrame.columns = edColumns
edDataFrame.head()

,브랜드,상호,주소,시도,군구,위도,경도
0,이디야,4.19사거리점,서울 강북구 삼양로 501 (수유동),서울,강북구,37.647893174147,127.014231720461
1,이디야,가오리역점,서울 강북구 삼각산로 108 (수유동),서울,강북구,37.6406008108564,127.016163948133
2,이디야,미아꿈의숲해링턴점,"서울 강북구 오현로 45 (미아동, 꿈의숲 해링턴 플레이스)",서울,강북구,37.6169375235603,127.034460650104
3,이디야,미아역점,서울 강북구 도봉로 207 -3(미아동),서울,강북구,37.6270583442503,127.02547799375
4,이디야,미아중앙점,서울 강북구 도봉로 128 (미아동),서울,강북구,37.6204702944714,127.028752360124


In [178]:
print('주소 변환에 문제가 있던 항목들')
incorrect_address

주소 변환에 문제가 있던 항목들


['서울 동작구 사당로16가길 96 (사당동) 1,2층', '서울 영등포구 영등포로35길 19 (영등포동6가)']

In [179]:
# 콤마를 공백으로, 2칸 공백은 1칸으로 치환합니다.
# 이후 공백으로 분리후, 슬라이싱을 사용하여 0번째 부터 3번째 글자까지만 추출합니다.
correct_address = [addr.replace(',', ' ').replace('  ', ' ').split(' ')[0:4] for addr in incorrect_address]
correct_address

[['서울', '동작구', '사당로16가길', '96'], ['서울', '영등포구', '영등포로35길', '19']]

In [180]:
# split() 함수로 나누어진 주소를 join() 함수로 다시 문자열을 합칩니다.
correct_address = [' '.join(addr) for addr in correct_address]
correct_address

['서울 동작구 사당로16가길 96', '서울 영등포구 영등포로35길 19']

In [181]:
# 보정이 된 주소 이름으로 Kakao API에게 위도/경도 추출을 요청합니다.
correction_geoinfo = [getGeoCoder(addr) for addr in correct_address]
correction_geoinfo

list index out of range


[('37.485107611205', '126.968823424325'), None]

In [182]:
edFilename = dataOut + 'ediya_file.csv'
edDataFrame.to_csv(edFilename, index=False, encoding='UTF-8')
print(f'{edFilename} 파일이 저장되었습니다.')

./../dataOut/ediya_file.csv 파일이 저장되었습니다.


In [183]:
# 위도, 경도 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {edDataFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {edDataFrame["경도"].isnull().sum()}')

위도 누락 데이터 갯수 : 2
경도 누락 데이터 갯수 : 2


In [184]:
# 할리스 매장

In [185]:
hollys_base_url = 'https://www.hollys.co.kr/store/korea/korStore2.do'
#                 'https://www.hollys.co.kr/store/korea/korStore2
#                 .do?pageNo=5&sido=&gugun=&store='

In [186]:
# 위도와 경도 추출에 문제가 있는 주소 정보들
incorrect_address = []

# 서비스 목록 이름을 저장해놓은 전역 변수 : 타입은 list
global_service_list = []

In [196]:
# 페이지 번호를 사용하여 매장들의 정보를 10개씩 읽어들이는 함수
def get_store_data(page_no):
    # parameters는 웹 페이지 요청 시 전송할 파라미터 사전
    # (우리는 '페이지 번호'와 '서울 지역'만 조회 예정)
    parameters = {
        'pageNo':page_no,
        'sido':'서울',
        'gugun':'',
        'store':''
    }

    # url에 데이터 요청 보내기
    response = requests.get(hollys_base_url,params = parameters)

    if response.status_code == 200:
        print(response.text)
        pass
    else:
        print(f'페이지 {page_no} 로딩에 실패하였습니다.')
        return []
    # end if

    soup = BeautifulSoup(response.text, 'html.parser')

    # <table> 태그의 'class' 속성이 'tb_store'인 요소를 찾습니다.
    store_table = soup.find('table',{'class':'tb_store'})

    all_store = store_table.find_all('tr')[1:] # 상단의 table header는 제외하기

    if len(all_store) == 0: # 더 이상 매장 정보가 없으면
        return None
    # end if

    stores = [] # 현재 페이지에서 반환될 모든 매장 정보

    for one_store in all_store: # 현재 페이지의 매장 정보들을 반복
        td_tags = one_store.find_all('td')

        if len(td_tags) == 0: # 더 이상 매장 정보가 없으면 무시
            continue
        # end if

        brand = '할리스'
        name = td_tags[1].text.strip()
        # 주소의 끝자락에 있는 "공백."을 치환
        address = td_tags[3].text.strip().replace(' .','')
        imsi = address.split(' ')
        sido = imsi[0]
        gungu = imsi[1]

        # 매장 서비스 목록 : 서비스 가능한 목록은 모두 <img> 태그로 만들어져 있습니다.
        # 기본 값으로 모든 매장에서 서비스되는 항목이 없다고 가정합니다.
        # DT : Drive Through
        service_dict = {'테라스':'no','흡연시설':'no','주차':'no','24시간':'no','DT 매장':'no'}

        store_service = td_tags[4]
        # print('-'*30)
        # print(store_service)

        service_img = store_service.find_all('img')
        # print('-'*30)
        # print(f'img 태그 갯수 : {len(service_img)}')

        # 매장에서 제공되는 서비스의 key 값을 'yes'로 변경합니다.
        for image in service_img:
            alt_tag = image.get('alt','')
            # print(alt_tag)
            service_dict[alt_tag] = 'yes'
        #end for

        # 매장 서비스에 대한 사전을 정렬시켜, 각 key들의 값 정보를 추출합니다.
        service_list = sorted(service_dict.keys())
        service_data = [service_dict[key] for key in service_list]
        print(service_data) # 이 항목은 stores 리스트에 추가

        global global_service_list # 전역 변수
        global_service_list = service_list

        phone_number = td_tags[5].text.strip()
        if phone_number == '.':
            phone_number = None

        # 위도와 경도에 대한 정보는 없습니다.
        # 그래서, 주소 정보와 Kakao API를 사용하여 구해야합니다.
        geo_info = getGeoCoder(address)

        if geo_info != None:
            latitude, longitude = geo_info

        else: # 위도와 경도 변환이 안되는 주소들
            incorrect_address.append(address)
            latitude = None
            longitude = None

        # list에 + 기호를 사용하면, 요소들을 합쳐줌.
        stores.append([brand, name, address, sido, gungu, latitude, longitude, phone_number] + service_data)
    # end for

    return stores

# end def get_store_data

In [197]:
hollysData = [] # 매장 정보를 저장할 리스트
page_no = 1 #페이지 번호를 +1씩 증가하면서 크롤링

while True:
    print(f'현재 {page_no} 페이지를 크롤링 중입니다.')
    stores = get_store_data(page_no) # 페이지당 10개

    # if not stores:
    if page_no == 5:
        print(f'더이상 매장 정보가 없습니다. 빠져나갑니다')
        break
    # end if

    for one in stores:
        hollysData.append(one)
    # end for
    page_no += 1
# end while True

현재 1 페이지를 크롤링 중입니다.
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">
<head>
	<title>할리스</title>
	<meta charset="UTF-8">

	<meta name="Subject" content="HOLLYS" />
	<meta name="Title" content="HOLLYS" />
	<meta name="Keywords" content="HOLLYS,HOLLYS,할리스,할리스,커피" />
	<meta name="Description" content="할리스는 1998년 국내 첫 에스프레소 커피전문점을 개점한 순수 국내브랜드로서 당당하게 시장 선점이라는 확고한 위치를 확보하고 국내 에스프레소 커피시장을 주도해 나가고 있습니다." />
	<meta name="Author" content="HOLLYS F&B" />
	<meta name="Publisher" content="HOLLYS F&B" />
	<meta name="Classification" content="COFFEE,커피,음료,푸드" />
	<meta name="Location" content="Korea" />
	<meta name="Author-Date" content="2015.04.01" />
	<meta name="Date" content="2025.12.08" />
	<meta name="Distribution" content="HOLLYS, HOLLYS F&B" />
	<meta name="Copyright" content="HOLLYS" />
	
	<meta property="og:type" content="website">
	<meta property="og:title" content="할리스">
	<meta property="og:description" content="HOLLYS">
	<meta property="og:url" content=

In [198]:
print(f'총 매장 갯수 : {len(hollysData)}')
hollysFrame = pd.DataFrame(hollysData)

총 매장 갯수 : 40


In [199]:
column_list = ['브랜드','상호','주소','시도','군구','위도','경도','전화 번호'] + global_service_list
hollysFrame.columns = column_list
hollysFrame.head()

,브랜드,상호,주소,시도,군구,위도,경도,전화 번호,24시간,DT 매장,주차,테라스,흡연시설
0,할리스,충무로역점,"서울특별시 중구 퇴계로36길 2 (필동2가, 동국대학교충무로영상센터)",서울특별시,중구,37.5606439510409,126.993567232255,02-2268-8960,no,no,yes,no,no
1,할리스,FKI타워2호점,"서울특별시 영등포구 여의대로 24 (여의도동, 에프케이아이타워) 1층",서울특별시,영등포구,37.5221803249647,126.919916550827,02-782-3647,no,no,no,no,no
2,할리스,신대방삼거리역점,서울특별시 동작구 상도로 60 (대방동) 1층~2층,서울특별시,동작구,37.4995614234716,126.927153186125,02-823-2377,no,no,no,no,no
3,할리스,여의도포스트타워점,서울특별시 영등포구 여의나루로 60 포스트타워 여의도 1층,서울특별시,영등포구,37.5223003029771,126.926225192863,02-2135-5321,no,no,yes,no,no
4,할리스,대림역점,서울특별시 영등포구 도림로 140 대림빌딩 101호,서울특별시,영등포구,37.4930241705718,126.897628371815,02-834-1000,no,no,no,yes,no


In [200]:
# 변환에 문제가 있었던 주소들
incorrect_address

['서울시 강북구 삼양로 547 1~3층',
 '서울특별시 강서구 등촌로 229 (등촌동) 1~2층',
 '서울특별시 송파구 석촌호수로 130 (삼전동, 신원빌딩) 삼전동 1-1외 3필지',
 '서울특별시 구로구 새말로 97 (신도림테크노마트) 104~106호',
 '서울특별시 금천구 가산디지털1로 146 109,110,111호',
 '서울특별시 서대문구 홍제내길 52,1~2층 (홍제동 361-49)',
 '서울특별시 동작구 흑석로 77 (흑석동) 1~4층',
 '서울특별시 종로구 성균관로 6 (명륜2가 204) 1~4층',
 '서울시 강북구 삼양로 547 1~3층',
 '서울특별시 강서구 등촌로 229 (등촌동) 1~2층',
 '서울특별시 송파구 석촌호수로 130 (삼전동, 신원빌딩) 삼전동 1-1외 3필지',
 '서울특별시 구로구 새말로 97 (신도림테크노마트) 104~106호',
 '서울특별시 금천구 가산디지털1로 146 109,110,111호',
 '서울특별시 서대문구 홍제내길 52,1~2층 (홍제동 361-49)',
 '서울특별시 동작구 흑석로 77 (흑석동) 1~4층',
 '서울특별시 종로구 성균관로 6 (명륜2가 204) 1~4층']

In [201]:
# 각 주소에 대하여 콤마를 스페이스로 치환하고, 공백 2칸도 1칸으로 치환합니다.
# split() 함수를 사용하여 list로 변경 후, 슬라이싱으로 0번째 부터 3번째까지 추출
correction_address = [addr.replace(',',' ').replace('  ',' ').split(' ')[0:4] for addr in incorrect_address]
correction_address

[['서울시', '강북구', '삼양로', '547'],
 ['서울특별시', '강서구', '등촌로', '229'],
 ['서울특별시', '송파구', '석촌호수로', '130'],
 ['서울특별시', '구로구', '새말로', '97'],
 ['서울특별시', '금천구', '가산디지털1로', '146'],
 ['서울특별시', '서대문구', '홍제내길', '52'],
 ['서울특별시', '동작구', '흑석로', '77'],
 ['서울특별시', '종로구', '성균관로', '6'],
 ['서울시', '강북구', '삼양로', '547'],
 ['서울특별시', '강서구', '등촌로', '229'],
 ['서울특별시', '송파구', '석촌호수로', '130'],
 ['서울특별시', '구로구', '새말로', '97'],
 ['서울특별시', '금천구', '가산디지털1로', '146'],
 ['서울특별시', '서대문구', '홍제내길', '52'],
 ['서울특별시', '동작구', '흑석로', '77'],
 ['서울특별시', '종로구', '성균관로', '6']]

In [202]:
# split() 함수론 나누어진 주소를 취합하여 보정된 주소로 변환합니다.
correction_address = [' '.join(addr) for addr in correction_address]
correction_address

['서울시 강북구 삼양로 547',
 '서울특별시 강서구 등촌로 229',
 '서울특별시 송파구 석촌호수로 130',
 '서울특별시 구로구 새말로 97',
 '서울특별시 금천구 가산디지털1로 146',
 '서울특별시 서대문구 홍제내길 52',
 '서울특별시 동작구 흑석로 77',
 '서울특별시 종로구 성균관로 6',
 '서울시 강북구 삼양로 547',
 '서울특별시 강서구 등촌로 229',
 '서울특별시 송파구 석촌호수로 130',
 '서울특별시 구로구 새말로 97',
 '서울특별시 금천구 가산디지털1로 146',
 '서울특별시 서대문구 홍제내길 52',
 '서울특별시 동작구 흑석로 77',
 '서울특별시 종로구 성균관로 6']

In [203]:
# 보정이 된 주소 이름으로 getGeoCoder 함수를 호출하여 위도와 경도를 추출해냄.
correction_geoInfo = [getGeoCoder(addr) for addr in correction_address]
correction_geoInfo

[('37.6516375000425', '127.012463182729'),
 ('37.5502242501914', '126.863458548999'),
 ('37.5071964496082', '127.091386305975'),
 ('37.5070438414784', '126.890227293528'),
 ('37.4787027326644', '126.883148970201'),
 ('37.584802093985', '126.936166344795'),
 ('37.5069826807571', '126.958246364678'),
 ('37.5835462396105', '126.998405014649'),
 ('37.6516375000425', '127.012463182729'),
 ('37.5502242501914', '126.863458548999'),
 ('37.5071964496082', '127.091386305975'),
 ('37.5070438414784', '126.890227293528'),
 ('37.4787027326644', '126.883148970201'),
 ('37.584802093985', '126.936166344795'),
 ('37.5069826807571', '126.958246364678'),
 ('37.5835462396105', '126.998405014649')]

In [206]:
# 변수 correction_data는 (주소, 위도, 경도) 형식의 tuple을 저장하고 있는 list 타입
correction_data = [(incorrect_address[idx], correction_geoInfo[idx][0], correction_geoInfo[idx][1])
    for idx in range(len(incorrect_address))
]
correction_data

[('서울시 강북구 삼양로 547 1~3층', '37.6516375000425', '127.012463182729'),
 ('서울특별시 강서구 등촌로 229 (등촌동) 1~2층', '37.5502242501914', '126.863458548999'),
 ('서울특별시 송파구 석촌호수로 130 (삼전동, 신원빌딩) 삼전동 1-1외 3필지',
  '37.5071964496082',
  '127.091386305975'),
 ('서울특별시 구로구 새말로 97 (신도림테크노마트) 104~106호',
  '37.5070438414784',
  '126.890227293528'),
 ('서울특별시 금천구 가산디지털1로 146 109,110,111호',
  '37.4787027326644',
  '126.883148970201'),
 ('서울특별시 서대문구 홍제내길 52,1~2층 (홍제동 361-49)',
  '37.584802093985',
  '126.936166344795'),
 ('서울특별시 동작구 흑석로 77 (흑석동) 1~4층', '37.5069826807571', '126.958246364678'),
 ('서울특별시 종로구 성균관로 6 (명륜2가 204) 1~4층', '37.5835462396105', '126.998405014649'),
 ('서울시 강북구 삼양로 547 1~3층', '37.6516375000425', '127.012463182729'),
 ('서울특별시 강서구 등촌로 229 (등촌동) 1~2층', '37.5502242501914', '126.863458548999'),
 ('서울특별시 송파구 석촌호수로 130 (삼전동, 신원빌딩) 삼전동 1-1외 3필지',
  '37.5071964496082',
  '127.091386305975'),
 ('서울특별시 구로구 새말로 97 (신도림테크노마트) 104~106호',
  '37.5070438414784',
  '126.890227293528'),
 ('서울특별시 금천구 가산디지털1로 146 109

In [207]:
# 반복문을 사용하여 실제 데이터 프레임에 위도와 경도 정보를 업데이트 합니다.
for onerow in correction_data:
    hollysFrame.loc[hollysFrame['주소'] == onerow[0],['위도','경도']] = [onerow[1],onerow[2]]

hsFilename = dataOut + 'hollys_file.csv'
hollysFrame.to_csv(hsFilename,index=False, encoding='UTF-8')
print(f'{hsFilename} 파일이 저장되었습니다.')

./../dataOut/hollys_file.csv 파일이 저장되었습니다.


In [208]:
# 위도, 경도 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {hollysFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {hollysFrame["경도"].isnull().sum()}')

위도 누락 데이터 갯수 : 0
경도 누락 데이터 갯수 : 0
